# Introduction
I constructed this Kernel from [MichaelMayer](https://www.kaggle.com/mayer79)'s great [RNN Starter Kernel](https://www.kaggle.com/mayer79/rnn-starter/code) code. Thanks for that! I've been using a similar Kernel with a Notebook but it's got plenty of other stuff in there which makes it more than just bare bones. Since this is not really my work, I figured I would share this Kernel with the community. It's meant to be MichaelMayer's starter code converted to a Notebook and nothing more. I had to make some changes, which I've indicated in the code.

# Basic idea of the Kernel
At test time, we will see a time series x of length 150 000 to predict the next earthquake. The idea of this kernel is to split x into contiguous chunks of size 1000 and feed summary statistics calculated from these chunks into a current neural net with 150 000 / 1000 = 150 time steps. 

# Validation
Validation is a very complex and crucial element of this competition. Here, the validation generator is fed with data until the second earthquake, while the training generator solely picks chunks after the second earthquake. In order to reach a better LB score, you will need to change this so that the training generator has access to the full data.

# Imports

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

from tqdm import tqdm

from scipy import stats
from scipy.signal import hann
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.signal import convolve

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, train_test_split
# Any results you write to the current directory are saved as output.

['unarch', 'test.zip', 'sample_submission.csv', 'train.csv.zip', 'test', 'train']


In [2]:
%matplotlib inline

# Setup

In [3]:
# Fix seeds
from numpy.random import seed
seed(639)
from tensorflow import set_random_seed
set_random_seed(5944)

#float_data = pd.read_csv("../input/train/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values
train_df = pd.read_csv('../input/train/train.csv', dtype={"acoustic_data": np.float32, "time_to_failure": np.float32})

In [4]:
rows = 150000
segments = int(np.floor(train_df.shape[0] / rows))
print("Number of segments: ", segments)

Number of segments:  4194


# Helper functions
## Helper function for data generator
Extracts mean, standard deviation, and quantiles per time step.
Can easily be extended. Expects a two dimensional array.

In [5]:
def extract_features(z):
     return np.c_[z.mean(axis=1), 
                  np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=1)),
                  z.std(axis=1)]

## Create features
For a given ending position "last_index", we split the last 150 000 values of *x* into 150 pieces of length 1000 each. So *n_steps * step_length* should equal 150 000.
From each piece, a set features are extracted. This results in a feature matrix of dimension *(150 time steps x features)*.  

In [6]:
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
       
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    # ORIGINAL: I changed this becuase I got an No OpKernel was registered to support Op 'CudnnRNN' error
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    # MY CHANGE: This doesn't fix things, I get the same errors
    temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1).astype(np.float32) - 5 ) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations. 
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]

## Generate features

In [7]:
# Query "create_X" to figure out the number of features
## n_features = create_X(float_data[0:150000]).shape[1]
## print("Our RNN is based on %i features"% n_features)
    
# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
    if max_index is None:
        max_index = len(data) - 1
     
    while True:
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        
        for j, row in enumerate(rows):
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            targets[j] = data[row - 1, 1]
        yield samples, targets
        
batch_size = 32

# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877
#float_data[second_earthquake, 1]

# Initialize generators
# train_gen = generator(float_data, batch_size=batch_size) # Use this for better score
## train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
## valid_gen = generator(float_data, batch_size=batch_size, max_index=second_earthquake)

In [8]:
#item = train_gen.__next__()

In [9]:
#print(type(item))

In [10]:
#print(item)

In [11]:
#train_arr, target_arr = item

In [12]:
#train_arr.shape

In [13]:
#target_arr.shape

In [14]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

def classic_sta_lta(x, length_sta, length_lta):
    sta = np.cumsum(x ** 2)
    # Convert to float
    sta = np.require(sta, dtype=np.float)
    # Copy for LTA
    lta = sta.copy()
    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta
    # Pad zeros
    sta[:length_lta - 1] = 0
    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny
    return sta / lta

In [15]:
train_X = pd.DataFrame(index=range(segments), dtype=np.float64)
train_y = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])
total_mean = train_df['acoustic_data'].mean()
total_std = train_df['acoustic_data'].std()
total_max = train_df['acoustic_data'].max()
total_min = train_df['acoustic_data'].min()
total_sum = train_df['acoustic_data'].sum()
total_abs_sum = np.abs(train_df['acoustic_data']).sum()

In [16]:
train_y.shape

(4194, 1)

In [17]:
def create_features(seg_id, seg, X):
    xc = pd.Series(seg['acoustic_data'].values)
    zc = np.fft.fft(xc)
    
    X.loc[seg_id, 'mean'] = xc.mean()
    X.loc[seg_id, 'std'] = xc.std()
    X.loc[seg_id, 'max'] = xc.max()
    X.loc[seg_id, 'min'] = xc.min()
    
    #FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X.loc[seg_id, 'Rmean'] = realFFT.mean()
    X.loc[seg_id, 'Rstd'] = realFFT.std()
    X.loc[seg_id, 'Rmax'] = realFFT.max()
    X.loc[seg_id, 'Rmin'] = realFFT.min()
    X.loc[seg_id, 'Imean'] = imagFFT.mean()
    X.loc[seg_id, 'Istd'] = imagFFT.std()
    X.loc[seg_id, 'Imax'] = imagFFT.max()
    X.loc[seg_id, 'Imin'] = imagFFT.min()
    X.loc[seg_id, 'Rmean_last_5000'] = realFFT[-5000:].mean()
    X.loc[seg_id, 'Rstd__last_5000'] = realFFT[-5000:].std()
    X.loc[seg_id, 'Rmax_last_5000'] = realFFT[-5000:].max()
    X.loc[seg_id, 'Rmin_last_5000'] = realFFT[-5000:].min()
    X.loc[seg_id, 'Rmean_last_15000'] = realFFT[-15000:].mean()
    X.loc[seg_id, 'Rstd_last_15000'] = realFFT[-15000:].std()
    X.loc[seg_id, 'Rmax_last_15000'] = realFFT[-15000:].max()
    X.loc[seg_id, 'Rmin_last_15000'] = realFFT[-15000:].min()
    
    X.loc[seg_id, 'mean_change_abs'] = np.mean(np.diff(xc))
    X.loc[seg_id, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0])
    X.loc[seg_id, 'abs_max'] = np.abs(xc).max()
    X.loc[seg_id, 'abs_min'] = np.abs(xc).min()
    
    X.loc[seg_id, 'std_first_50000'] = xc[:50000].std()
    X.loc[seg_id, 'std_last_50000'] = xc[-50000:].std()
    X.loc[seg_id, 'std_first_10000'] = xc[:10000].std()
    X.loc[seg_id, 'std_last_10000'] = xc[-10000:].std()
    
    X.loc[seg_id, 'avg_first_50000'] = xc[:50000].mean()
    X.loc[seg_id, 'avg_last_50000'] = xc[-50000:].mean()
    X.loc[seg_id, 'avg_first_10000'] = xc[:10000].mean()
    X.loc[seg_id, 'avg_last_10000'] = xc[-10000:].mean()
    
    X.loc[seg_id, 'min_first_50000'] = xc[:50000].min()
    X.loc[seg_id, 'min_last_50000'] = xc[-50000:].min()
    X.loc[seg_id, 'min_first_10000'] = xc[:10000].min()
    X.loc[seg_id, 'min_last_10000'] = xc[-10000:].min()
    
    X.loc[seg_id, 'max_first_50000'] = xc[:50000].max()
    X.loc[seg_id, 'max_last_50000'] = xc[-50000:].max()
    X.loc[seg_id, 'max_first_10000'] = xc[:10000].max()
    X.loc[seg_id, 'max_last_10000'] = xc[-10000:].max()
    
    X.loc[seg_id, 'max_to_min'] = xc.max() / np.abs(xc.min())
    X.loc[seg_id, 'max_to_min_diff'] = xc.max() - np.abs(xc.min())
    X.loc[seg_id, 'count_big'] = len(xc[np.abs(xc) > 500])
    X.loc[seg_id, 'sum'] = xc.sum()
    
    X.loc[seg_id, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(xc[:50000]) / xc[:50000][:-1]))[0])
    X.loc[seg_id, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(xc[-50000:]) / xc[-50000:][:-1]))[0])
    X.loc[seg_id, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(xc[:10000]) / xc[:10000][:-1]))[0])
    X.loc[seg_id, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(xc[-10000:]) / xc[-10000:][:-1]))[0])
    
    X.loc[seg_id, 'q95'] = np.quantile(xc, 0.95)
    X.loc[seg_id, 'q99'] = np.quantile(xc, 0.99)
    X.loc[seg_id, 'q05'] = np.quantile(xc, 0.05)
    X.loc[seg_id, 'q01'] = np.quantile(xc, 0.01)
    
    X.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X.loc[seg_id, 'trend'] = add_trend_feature(xc)
    X.loc[seg_id, 'abs_trend'] = add_trend_feature(xc, abs_values=True)
    X.loc[seg_id, 'abs_mean'] = np.abs(xc).mean()
    X.loc[seg_id, 'abs_std'] = np.abs(xc).std()
    
    X.loc[seg_id, 'mad'] = xc.mad()
    X.loc[seg_id, 'kurt'] = xc.kurtosis()
    X.loc[seg_id, 'skew'] = xc.skew()
    X.loc[seg_id, 'med'] = xc.median()
    
    X.loc[seg_id, 'Hilbert_mean'] = np.abs(hilbert(xc)).mean()
    X.loc[seg_id, 'Hann_window_mean'] = (convolve(xc, hann(150), mode='same') / sum(hann(150))).mean()
    X.loc[seg_id, 'classic_sta_lta1_mean'] = classic_sta_lta(xc, 500, 10000).mean()
    X.loc[seg_id, 'classic_sta_lta2_mean'] = classic_sta_lta(xc, 5000, 100000).mean()
    X.loc[seg_id, 'classic_sta_lta3_mean'] = classic_sta_lta(xc, 3333, 6666).mean()
    X.loc[seg_id, 'classic_sta_lta4_mean'] = classic_sta_lta(xc, 10000, 25000).mean()
    X.loc[seg_id, 'Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_1500_mean'] = xc.rolling(window=1500).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_3000_mean'] = xc.rolling(window=3000).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_6000_mean'] = xc.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X.loc[seg_id, 'exp_Moving_average_300_mean'] = (ewma(xc, span=300).mean()).mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_3000_mean'] = ewma(xc, span=3000).mean().mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_30000_mean'] = ewma(xc, span=6000).mean().mean(skipna=True)
    no_of_std = 2
    X.loc[seg_id, 'MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X.loc[seg_id,'MA_700MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X.loc[seg_id,'MA_700MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X.loc[seg_id, 'MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X.loc[seg_id,'MA_400MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id,'MA_400MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id, 'MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    
    X.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(xc, [75, 25]))
    X.loc[seg_id, 'q999'] = np.quantile(xc,0.999)
    X.loc[seg_id, 'q001'] = np.quantile(xc,0.001)
    X.loc[seg_id, 'ave10'] = stats.trim_mean(xc, 0.1)
    
    for windows in [10, 100, 1000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values
        
        X.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

In [18]:
for seg_id in tqdm_notebook(range(segments)):
    seg = train_df.iloc[seg_id*rows:seg_id*rows+rows]
    create_features(seg_id, seg, train_X)
    train_y.loc[seg_id, 'time_to_failure'] = seg['time_to_failure'].values[-1]

In [19]:
train_X.shape

(4194, 154)

In [20]:
train_X.head(10)

,mean,std,max,min,Rmean,Rstd,Rmax,Rmin,Imean,Istd,...,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,4.884113,5.101106,104.0,-98.0,1.200000e+01,2349.811482,732617.0,-20121.154171,-1.067140e-15,1399.854635,...,0.295715,5.629,3.896,4.072,4.379,5.338,5.484,-1.704698e-06,74222.343443,5.629
1,4.725767,6.588824,181.0,-154.0,5.000000e+00,2566.032248,708865.0,-31056.675076,7.033426e-16,1810.312266,...,0.231587,5.667,3.412,4.233,4.345,5.066,5.223,-2.449664e-06,74364.426478,5.667
2,4.906394,6.967397,140.0,-106.0,5.000000e+00,2683.549049,735959.0,-27654.557067,-1.261166e-15,1921.220576,...,0.267012,5.957,4.055,4.239,4.446,5.344,5.486,1.140940e-06,74594.984584,5.957
3,4.902240,6.922305,197.0,-199.0,5.000000e+00,2685.788525,735336.0,-25622.393604,-9.701277e-16,1891.826366,...,0.266700,5.858,3.722,4.304,4.433,5.317,5.453,-2.550336e-06,74333.143623,5.858
4,4.908720,7.301110,145.0,-126.0,1.200000e+01,2761.715771,736308.0,-26271.075117,2.910383e-16,1995.742969,...,0.228005,6.078,3.918,4.434,4.543,5.306,5.491,1.550336e-06,74428.435969,6.078
5,4.913513,5.434111,142.0,-144.0,1.400000e+01,2420.968091,737027.0,-24168.435450,-1.940255e-16,1479.700814,...,0.293941,5.838,4.042,4.270,4.422,5.402,5.620,-3.697987e-06,74539.909434,5.838
6,4.855660,5.687823,120.0,-78.0,-1.624964e-15,2437.523641,728349.0,-23876.893136,-1.940255e-16,1564.532358,...,0.219472,5.546,3.958,4.285,4.504,5.214,5.370,-1.140940e-07,74501.904751,5.546
7,4.505427,5.854512,139.0,-134.0,3.000000e+00,2361.259304,675814.0,-25427.499452,1.200533e-15,1615.715786,...,0.218514,5.180,3.489,4.040,4.148,4.863,4.953,-1.395973e-06,74515.926219,5.180
8,4.717834,7.789643,168.0,-156.0,1.000000e+00,2805.303460,707675.0,-34542.227581,4.486841e-16,2137.915924,...,0.240917,5.717,3.933,4.158,4.315,5.104,5.337,2.221477e-06,74464.251035,5.717
9,4.730960,6.890459,152.0,-126.0,5.000000e+00,2620.173754,709644.0,-23173.271872,1.940255e-16,1900.967780,...,0.258462,5.423,3.832,4.190,4.301,5.128,5.261,5.127517e-06,74344.216475,5.423


In [21]:
scaler = StandardScaler()
scaler.fit(train_X)
scaled_train_X = pd.DataFrame(scaler.transform(train_X), columns=train_X.columns)

In [22]:
scaled_train_X.head(10)

,mean,std,max,min,Rmean,Rstd,Rmax,Rmin,Imean,Istd,...,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,1.424140,-0.170214,-0.218194,0.193218,1.199316,-0.105007,1.424140,0.156736,-0.458570,-0.169052,...,0.268470,-0.004742,0.178278,0.287332,0.965402,1.509153,0.885262,-0.631300,-1.832422,-0.004742
1,0.805716,0.004734,0.063936,-0.018037,0.078886,-0.008599,0.805716,-0.222118,0.286705,0.007212,...,-0.141264,0.007341,-0.025387,0.622391,0.842747,0.522428,0.294357,-0.912054,-0.890022,0.007341
2,1.511156,0.049252,-0.086289,0.163039,0.078886,0.043800,1.511155,-0.104254,-0.540244,0.054839,...,0.085078,0.099556,0.245184,0.634878,1.207106,1.530919,0.889790,0.441128,0.639209,0.099556
3,1.494934,0.043950,0.122560,-0.187796,0.078886,0.044798,1.494934,-0.033851,-0.417733,0.042217,...,0.083085,0.068076,0.105059,0.770151,1.160208,1.432972,0.815078,-0.949994,-1.097513,0.068076
4,1.520242,0.088495,-0.067969,0.087590,1.199316,0.078652,1.520242,-0.056324,0.113148,0.086842,...,-0.164151,0.138032,0.187535,1.040695,1.557034,1.393068,0.901110,0.595416,-0.465464,0.138032
5,1.538962,-0.131054,-0.078961,0.019687,1.519439,-0.073279,1.538962,0.016521,-0.091037,-0.134763,...,0.257139,0.061716,0.239714,0.699393,1.120525,1.741324,1.193167,-1.382506,0.273910,0.061716
6,1.313016,-0.101219,-0.159569,0.268667,-0.721422,-0.065898,1.313016,0.026621,-0.091037,-0.098334,...,-0.218670,-0.031134,0.204367,0.730609,1.416341,1.059323,0.627166,-0.031853,0.021835,-0.031134
7,-0.054823,-0.081617,-0.089953,0.057411,-0.241237,-0.099902,-0.054824,-0.027099,0.495994,-0.076354,...,-0.224790,-0.147515,0.007014,0.220737,0.132067,-0.213989,-0.316925,-0.514952,0.114836,-0.147515
8,0.774733,0.145944,0.016304,-0.025582,-0.561360,0.098087,0.774732,-0.342873,0.179508,0.147895,...,-0.081653,0.023240,0.193847,0.466308,0.734522,0.660279,0.552453,0.848347,-0.227912,0.023240
9,0.825998,0.040205,-0.042321,0.087590,0.078886,0.015542,0.825999,0.050997,0.072311,0.046142,...,0.030453,-0.070246,0.151347,0.532904,0.684017,0.747343,0.380389,1.943539,-1.024070,-0.070246


In [23]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
test_X = pd.DataFrame(columns=train_X.columns, dtype=np.float64, index=submission.index)

In [ ]:
for seg_id in tqdm_notebook(test_X.index):
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    create_features(seg_id, seg, test_X)

In [52]:
scaled_test_X = pd.DataFrame(scaler.transform(test_X), columns=test_X.columns)

In [24]:
X_train, X_holdout_test, y_train, y_noldout_test = train_test_split(
    scaled_train_X,
    train_y,
    test_size = 1.0 / 6,
    random_state=0
)

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    test_size = 0.2,
    random_state=0
)

In [26]:
y_train.shape

(2796, 1)

In [46]:
y_holdout_test = y_noldout_test

In [47]:
y_holdout_test.shape

(699, 1)

In [29]:
x_train_values = X_train.values

In [30]:
x_train_values.shape

(2796, 154)

In [31]:
n_features = 154

In [ ]:
2516 / 4

In [32]:
y_valid.shape

(699, 1)

In [ ]:
def generator(data, target, batch_size=629, n_steps=4, n_features=154):
    while True:
        # Pick indices of ending positions
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        #samples = np.zeros((batch_size, batch_size, n_features))
        targets = np.zeros(batch_size, )
        
        for j in range(n_steps):
            samples[j] = data[j * batch_size:(j + 1) * batch_size, :]
            targets[j * batch_size:(j + 1) * batch_size] = target[j * batch_size:(j + 1) * batch_size]
        yield samples, targets

In [ ]:
#train_gen = generator(x_train_values, y_train.values.ravel())

In [ ]:
#valid_gen = generator(X_valid, y_valid.values.ravel(), n_steps=1)

In [ ]:
#train_gen.__next__()

# Define the model

In [34]:
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

cb = [ModelCheckpoint("model.hdf5", save_best_only=True, period=3)]

Using TensorFlow backend.


In [ ]:
#model = Sequential()
#model.add(CuDNNGRU(48, input_shape=(None, n_features)))
#model.add(Dense(10, activation='relu'))
#model.add(Dense(1))

#model.summary()

In [39]:
model_dense = Sequential()
#model_dense.add(CuDNNGRU(48, input_shape=(None, n_features)))
model_dense.add(Dense(154, input_shape=(154,)))
model_dense.add(Dense(308, activation='sigmoid'))
model_dense.add(Dense(308, activation='relu'))
model_dense.add(Dense(154, activation='sigmoid'))
model_dense.add(Dense(154, activation='relu'))
model_dense.add(Dense(77, activation='sigmoid'))
model_dense.add(Dense(77, activation='relu'))
model_dense.add(Dense(1))

model_dense.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 154)               23870     
_________________________________________________________________
dense_5 (Dense)              (None, 308)               47740     
_________________________________________________________________
dense_6 (Dense)              (None, 308)               95172     
_________________________________________________________________
dense_7 (Dense)              (None, 154)               47586     
_________________________________________________________________
dense_8 (Dense)              (None, 154)               23870     
_________________________________________________________________
dense_9 (Dense)              (None, 77)                11935     
_________________________________________________________________
dense_10 (Dense)             (None, 77)                6006      
__________

In [40]:
model_dense.compile(optimizer=adam(lr=0.0005), loss="mae")

# Compile and fit model

#model.compile(optimizer=adam(lr=0.0005), loss="mae")
'''
history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=0,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)
'''

from sklearn.model_selection import StratifiedShuffleSplit
loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)
callbacks_list = [EarlyStopping(monitor='val_auc', patience=10,mode='max'),loss_history, annealer]
sss = StratifiedShuffleSplit(n_splits=10)
for train_index, test_index in sss.split(train_features, train_targets):
    X_train, X_val = train_features[train_index], train_features[test_index]
    Y_train, Y_val = train_targets[train_index], train_targets[test_index]
    print("{} iteration".format(i+1))
    history= model.fit(X_train,Y_train,batch_size=512,epochs=50,callbacks=callbacks_list,verbose=1,validation_data=(X_val,Y_val))
    del X_train, X_val, Y_train, Y_val
    gc.collect()

In [44]:
fit_result = model_dense.fit(
    X_train,
    y_train,
    #steps_per_epoch=1000,
    batch_size=699,
    epochs=1000,
    #callbacks=callbacks_list,
    callbacks=cb,
    verbose=1,
    validation_data=(X_valid, y_valid),
    #validation_steps=200
)

Train on 2796 samples, validate on 699 samples
Epoch 1/1000
2796/2796 [==============================] - 0s 14us/step - loss: 0.5787 - val_loss: 2.5615
Epoch 2/1000
2796/2796 [==============================] - 0s 14us/step - loss: 0.5414 - val_loss: 2.6152
Epoch 3/1000
2796/2796 [==============================] - 0s 13us/step - loss: 0.5317 - val_loss: 2.6090
Epoch 4/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.5889 - val_loss: 2.6051
Epoch 5/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.5811 - val_loss: 2.5995
Epoch 6/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.5329 - val_loss: 2.6389
Epoch 7/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.5141 - val_loss: 2.5695
Epoch 8/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.5450 - val_loss: 2.6475
Epoch 9/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.5391 - val_loss: 2.6125
Epoch 10

Epoch 78/1000
2796/2796 [==============================] - 0s 14us/step - loss: 0.5718 - val_loss: 2.5472
Epoch 79/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.6382 - val_loss: 2.5738
Epoch 80/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.6677 - val_loss: 2.5538
Epoch 81/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.6336 - val_loss: 2.5619
Epoch 82/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.5462 - val_loss: 2.5573
Epoch 83/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.4786 - val_loss: 2.6169
Epoch 84/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.4538 - val_loss: 2.6064
Epoch 85/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.4397 - val_loss: 2.6157
Epoch 86/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.4346 - val_loss: 2.5941
Epoch 87/1000
2796/2796 [=====================

Epoch 155/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.3903 - val_loss: 2.6089
Epoch 156/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.3916 - val_loss: 2.6133
Epoch 157/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.4289 - val_loss: 2.6266
Epoch 158/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.4250 - val_loss: 2.6297
Epoch 159/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.4236 - val_loss: 2.6377
Epoch 160/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.4228 - val_loss: 2.6156
Epoch 161/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.3737 - val_loss: 2.6116
Epoch 162/1000
2796/2796 [==============================] - 0s 13us/step - loss: 0.3977 - val_loss: 2.6023
Epoch 163/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.3818 - val_loss: 2.5998
Epoch 164/1000
2796/2796 [===========

2796/2796 [==============================] - 0s 10us/step - loss: 0.3842 - val_loss: 2.6211
Epoch 232/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.3874 - val_loss: 2.6399
Epoch 233/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.4487 - val_loss: 2.6113
Epoch 234/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.4938 - val_loss: 2.6393
Epoch 235/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.6126 - val_loss: 2.6710
Epoch 236/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.5661 - val_loss: 2.6880
Epoch 237/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.5347 - val_loss: 2.6591
Epoch 238/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.4967 - val_loss: 2.6033
Epoch 239/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.4906 - val_loss: 2.6455
Epoch 240/1000
2796/2796 [==========================

2796/2796 [==============================] - 0s 12us/step - loss: 0.4557 - val_loss: 2.5731
Epoch 308/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.5151 - val_loss: 2.6063
Epoch 309/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.5577 - val_loss: 2.5951
Epoch 310/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.3554 - val_loss: 2.6041
Epoch 311/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.3667 - val_loss: 2.6339
Epoch 312/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.3337 - val_loss: 2.6320
Epoch 313/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.3014 - val_loss: 2.6630
Epoch 314/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.3133 - val_loss: 2.6145
Epoch 315/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.3332 - val_loss: 2.6506
Epoch 316/1000
2796/2796 [==========================

...

2796/2796 [==============================] - 0s 10us/step - loss: 0.2720 - val_loss: 2.6481
Epoch 688/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2591 - val_loss: 2.6538
Epoch 689/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2907 - val_loss: 2.6148
Epoch 690/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.2844 - val_loss: 2.6440
Epoch 691/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.2322 - val_loss: 2.6490
Epoch 692/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2136 - val_loss: 2.6487
Epoch 693/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.2274 - val_loss: 2.6510
Epoch 694/1000
2796/2796 [==============================] - 0s 12us/step - loss: 0.2336 - val_loss: 2.6269
Epoch 695/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2728 - val_loss: 2.6190
Epoch 696/1000
2796/2796 [==========================

2796/2796 [==============================] - 0s 11us/step - loss: 0.2572 - val_loss: 2.6305
Epoch 764/1000
2796/2796 [==============================] - 0s 13us/step - loss: 0.2332 - val_loss: 2.6443
Epoch 765/1000
2796/2796 [==============================] - 0s 13us/step - loss: 0.2430 - val_loss: 2.6561
Epoch 766/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2557 - val_loss: 2.6168
Epoch 767/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2157 - val_loss: 2.6314
Epoch 768/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.1916 - val_loss: 2.6409
Epoch 769/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.1948 - val_loss: 2.6468
Epoch 770/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.2070 - val_loss: 2.6438
Epoch 771/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.1806 - val_loss: 2.6668
Epoch 772/1000
2796/2796 [==========================

2796/2796 [==============================] - 0s 12us/step - loss: 0.2142 - val_loss: 2.6278
Epoch 840/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.1975 - val_loss: 2.6532
Epoch 841/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.2181 - val_loss: 2.6301
Epoch 842/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.2451 - val_loss: 2.6395
Epoch 843/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2184 - val_loss: 2.6312
Epoch 844/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2091 - val_loss: 2.6478
Epoch 845/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1857 - val_loss: 2.6352
Epoch 846/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2497 - val_loss: 2.6514
Epoch 847/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1924 - val_loss: 2.6378
Epoch 848/1000
2796/2796 [==========================

2796/2796 [==============================] - 0s 10us/step - loss: 0.2367 - val_loss: 2.6602
Epoch 916/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2732 - val_loss: 2.6286
Epoch 917/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2053 - val_loss: 2.6466
Epoch 918/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2267 - val_loss: 2.6637
Epoch 919/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2473 - val_loss: 2.6396
Epoch 920/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1947 - val_loss: 2.6492
Epoch 921/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1724 - val_loss: 2.6428
Epoch 922/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.1647 - val_loss: 2.6639
Epoch 923/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1657 - val_loss: 2.6643
Epoch 924/1000
2796/2796 [==========================

2796/2796 [==============================] - 0s 10us/step - loss: 0.3443 - val_loss: 2.6302
Epoch 992/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2792 - val_loss: 2.6112
Epoch 993/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.2348 - val_loss: 2.6489
Epoch 994/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1984 - val_loss: 2.6453
Epoch 995/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1821 - val_loss: 2.6340
Epoch 996/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1651 - val_loss: 2.6337
Epoch 997/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.1632 - val_loss: 2.6484
Epoch 998/1000
2796/2796 [==============================] - 0s 11us/step - loss: 0.1760 - val_loss: 2.6529
Epoch 999/1000
2796/2796 [==============================] - 0s 10us/step - loss: 0.2219 - val_loss: 2.6424
Epoch 1000/1000
2796/2796 [=========================

In [48]:
print(mean_absolute_error(y_holdout_test, model_dense.predict(X_holdout_test)))

2.5385580442543376


In [49]:
from keras.models import load_model

In [50]:
saved_model_dense = load_model('model.hdf5')

In [51]:
print(mean_absolute_error(y_holdout_test, saved_model_dense.predict(X_holdout_test)))

2.049791208451723


In [53]:
test_prediction = saved_model_dense.predict(scaled_test_X)

fit_results = model.fit_generator(
    train_gen,
    steps_per_epoch=1000,
    epochs=30,
    verbose=1,
    callbacks=cb,
    validation_data=valid_gen,
    validation_steps=200
)

# Visualize accuracies

import matplotlib.pyplot as plt

def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None

perf_plot(history)

# Load submission file

submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

## Prepare submission data
Load each test data, create the feature matrix, get numeric prediction


for i, seg_id in enumerate(tqdm(submission.index)):
    print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

In [54]:
submission.time_to_failure = test_prediction

In [55]:
submission.head()

,time_to_failure
seg_id,
seg_00030f,7.636238
seg_0012b5,7.636238
seg_00184e,7.636238
seg_003339,7.636238
seg_0042cc,7.636238


## Save submission file

In [56]:
submission.to_csv('rnn-submission.csv')